## 🧠 Natural Language to SQL Converter using Groq API

This notebook demonstrates how to convert Natural Language (NL) queries into SQL using the **Groq API** and the **LLaMA3-70B model**.

### 🔍 Objective
Build an interface that accepts natural language input and returns SQL queries using a large language model (LLM). This supports rapid query generation without needing users to know SQL syntax.

---

### **🚀 Key Features**
- Uses the **Groq API** to access the LLaMA3-70B model
- Supports live NL-to-SQL query conversion
- Can be deployed via Hugging Face or run locally


In [ ]:
# Install required packages
!pip install groq pandas numpy scikit-learn gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00


# 🔑 API Key Cell
# 🔐 Set Your Groq API Key
Ensure you have a valid [Groq API key](https://console.groq.com/) to use the LLaMA3-70B model.

In [ ]:
import os

# 🔐 Set your Groq API Key here
os.environ['GROQ_API_KEY'] = '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'  # <-- Replace this with your real key

# 🚀 Initialization Message
print("🚀 Initializing NL2SQL Application with Groq API...")

# ⚠️ Red-colored warning if API key is not found
if not os.environ.get('GROQ_API_KEY'):
    print("\033[91m⚠️  WARNING: GROQ_API_KEY environment variable not set!\033[0m")
    print("\033[91mPlease set your Groq API key: export GROQ_API_KEY='your_key_here'\033[0m")
else:
    print("✅ API Key found. Ready to use Groq API.")

# -------------------------------------------------
# 🧠 Your NL2SQL Application logic starts below
# -------------------------------------------------

# Example: How you might use the key (pseudo-code)
api_key = os.environ['GROQ_API_KEY']

# Example Function
def query_groq_nl2sql(natural_language_query):
    # Replace with actual Groq API call code
    print(f"🧠 Processing your query using Groq: \"{natural_language_query}\"")
    # Dummy SQL Output
    sql_output = f"SELECT * FROM users WHERE name = 'John';"
    return sql_output

# Example usage
user_input = "Show me all users named John"
sql_query = query_groq_nl2sql(user_input)
print("🧾 Generated SQL Query:")
print(sql_query)


🚀 Initializing NL2SQL Application with Groq API...
✅ API Key found. Ready to use Groq API.
🧠 Processing your query using Groq: "Show me all users named John"
🧾 Generated SQL Query:
SELECT * FROM users WHERE name = 'John';


## 💻 Full App: Natural Language to SQL Converter using Groq API

This cell integrates all the components:
- Groq API for LLaMA3-70B model-based text-to-SQL conversion
- SQLite for query execution on an employee dataset
- Gradio for an interactive web UI

In [ ]:
# -*- coding: utf-8 -*-
"""GR SQL App Full with Groq API Integration"""

import os
import pandas as pd
import sqlite3
import numpy as np
import json
import re
from typing import List, Dict, Tuple
from groq import Groq
import gradio as gr
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# ------------------------------
# ✅ STEP 1: INSERT GROQ API KEY HERE
# ------------------------------
os.environ["GROQ_API_KEY"] = "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"  # 🔴 <--- INSERT API KEY HERE

# ------------------------------
# SQL Converter Using Groq API
# ------------------------------

class EnhancedNL2SQLConverter:
    def __init__(self, model_name: str = "llama3-70b-8192"):
        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
        self.model_name = model_name
        print(f"Using Groq API with model: {self.model_name}")

        self.default_schema = """
        Table: employees
        Columns:
        - id (INTEGER) PRIMARY KEY
        - name (TEXT) NOT NULL
        - department (TEXT)
        - salary (REAL)
        - hire_date (TEXT)
        - manager_id (INTEGER)
        """

    def generate_sql(self, query: str, schema: str = None) -> str:
        try:
            schema_to_use = schema or self.default_schema

            system_prompt = """You are an expert SQL query generator. Convert natural language questions to SQL queries based on the provided database schema.

Rules:
1. Only return the SQL query, nothing else
2. Use proper SQL syntax
3. Be precise with column names and table names
4. Use appropriate WHERE clauses, JOINs, and aggregations as needed
5. For date comparisons, use proper date format
6. Don't include explanations, just the SQL query"""

            user_prompt = f"""Database Schema:
{schema_to_use}

Natural Language Question: {query}

Generate the SQL query:"""

            chat_completion = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=200
            )

            sql_query = chat_completion.choices[0].message.content.strip()
            return self._clean_sql(sql_query)

        except Exception as e:
            print(f"Error generating SQL: {str(e)}")
            return "ERROR: Could not generate SQL query"

    def _clean_sql(self, sql: str) -> str:
        sql = sql.strip()
        sql = re.sub(r'```sql\n?', '', sql)
        sql = re.sub(r'```\n?', '', sql)
        sql = re.sub(r'^["\']|["\']$', '', sql)
        sql = sql.rstrip(';')

        sql_keywords = ['SELECT', 'INSERT', 'UPDATE', 'DELETE', 'CREATE', 'DROP', 'ALTER']
        if not any(sql.upper().startswith(keyword) for keyword in sql_keywords):
            for keyword in sql_keywords:
                if keyword in sql.upper():
                    sql = sql[sql.upper().find(keyword):]
                    break
        return sql

# ------------------------------
# SQL Evaluator & Test Database
# ------------------------------

class SQLEvaluator:
    def __init__(self):
        self.db_path = "test_database.db"
        self.setup_test_database()

    def setup_test_database(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            department TEXT,
            salary REAL,
            hire_date TEXT,
            manager_id INTEGER
        )''')
        sample_data = [
            (1, 'Alice Johnson', 'Engineering', 75000, '2022-01-15', None),
            (2, 'Bob Smith', 'Sales', 65000, '2021-06-20', None),
            (3, 'Charlie Brown', 'Engineering', 80000, '2020-03-10', 1),
            (4, 'Diana Prince', 'HR', 60000, '2023-02-28', None),
            (5, 'Eve Wilson', 'Sales', 70000, '2022-11-05', 2),
            (6, 'Frank Miller', 'Engineering', 85000, '2019-08-12', 1),
            (7, 'Grace Lee', 'Marketing', 55000, '2023-01-20', None),
            (8, 'Henry Davis', 'Engineering', 72000, '2022-07-30', 1)
        ]
        cursor.executemany('''
        INSERT OR REPLACE INTO employees (id, name, department, salary, hire_date, manager_id)
        VALUES (?, ?, ?, ?, ?, ?)''', sample_data)
        conn.commit()
        conn.close()

    def execute_sql(self, sql_query: str) -> Tuple[bool, any]:
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute(sql_query)

            if sql_query.strip().upper().startswith('SELECT'):
                results = cursor.fetchall()
                columns = [description[0] for description in cursor.description]
                conn.close()
                return True, {'columns': columns, 'data': results}
            else:
                conn.commit()
                conn.close()
                return True, "Query executed successfully"
        except Exception as e:
            return False, str(e)

# ------------------------------
# Gradio UI
# ------------------------------

converter = EnhancedNL2SQLConverter()
evaluator = SQLEvaluator()

def process_nl_query(nl_query: str) -> Tuple[str, str]:
    generated_sql = converter.generate_sql(nl_query)
    success, result = evaluator.execute_sql(generated_sql)

    if success and isinstance(result, dict):
        df = pd.DataFrame(result['data'], columns=result['columns'])
        return generated_sql, df.to_markdown(index=False)
    elif success:
        return generated_sql, result
    else:
        return generated_sql, f"Error executing query:\n{result}"

iface = gr.Interface(
    fn=process_nl_query,
    inputs=gr.Textbox(label="Enter Natural Language Query"),
    outputs=[
        gr.Textbox(label="Generated SQL Query"),
        gr.Textbox(label="Query Output")
    ],
    title="NL2SQL with Groq + SQLite",
    description="Enter a natural language query about the employee database. The app will generate SQL using the Groq API and show the results."
)

if __name__ == "__main__":
    iface.launch()


Using Groq API with model: llama3-70b-8192
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5b1e9ac433538b0318.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
